## Librerías

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
from pyspark.sql.functions import to_date

## Configuracion de Storage Account

In [0]:
access_key = "iBjJ9xc/hIfQyzVS+q74rigNxoyegk0qCO0xwLjs5HH6rwCNzczsv610q6QKHtySstLAv1H2cF0v+AStfFQvxg=="
spark.conf.set("fs.azure.account.key.dataintegrador.dfs.core.windows.net", access_key)

## Paths

In [0]:
raw_data = "abfss://integrador-longas-elias@dataintegrador.dfs.core.windows.net/rawdata"

In [0]:
artistPATH = raw_data + '/azuresqldb/dbo.artistDf' 
billboardHot100PATH = raw_data + '/azuresqldb/dbo.billboardHot100_1999_2019'
grammyAlbumsPATH = raw_data + '/azuresqldb/dbo.grammyAlbums_1999_2019'
grammySongsPATH = raw_data + '/azuresqldb/dbo.grammySongs_1999_2019'
riaaAlbumCertsPATH = raw_data + '/azuresqldb/dbo.riaaAlbumCerts_1999_2019'
riaaSingleCertsPATH = raw_data + '/azuresqldb/dbo.riaaSingleCerts_1999_2019'
songAttributesPATH = raw_data + '/azuresqldb/dbo.songAttributes_1999_2019'
spotifyWeeklyTop200PATH = raw_data + '/azuresqldb/dbo.spotifyWeeklyTop200Streams'

### Carga de Datasets

In [0]:
artist = spark.read.option("header",True).csv(artistPATH)
bbHot100 = spark.read.option("header",True).csv(billboardHot100PATH)
grammyAlbums = spark.read.option("header",True).csv(grammyAlbumsPATH)
grammySongs = spark.read.option("header",True).csv(grammySongsPATH)
riaaAlbumCerts = spark.read.option("header",True).csv(riaaAlbumCertsPATH)
riaaSingleCerts = spark.read.option("header",True).csv(riaaSingleCertsPATH)
songAttributes = spark.read.option("header",True).csv(songAttributesPATH)
spotifyWeeklyTop200 = spark.read.option("header",True).csv(spotifyWeeklyTop200PATH)

## Transformaciones de Dataframes
Al menos 3 transformaciones a elección (limpieza/ borrado de duplicados/ casteos / agregar o eliminar columnas)

### Artist

In [0]:
#artist.display()

In [0]:
#artist.printSchema()

In [0]:
#Eliminando la columna "X"
artist = artist.drop("X") 

In [0]:
# Borrando registros duplicados si los hay
artist = artist.dropDuplicates()

In [0]:
#Casteando columnas de str a int
artist = artist.withColumn("Followers", col("Followers").cast("int")) 
artist = artist.withColumn("NumAlbums", col("NumAlbums").cast("int"))

In [0]:
# Reemplazando todos los nulos por "0" o "" respectivamente
artist = artist.fillna(value = 0).fillna("")

In [0]:
# Cambiando de nombre a la columna Group.Solo por Group_Solo
artist = artist.withColumnRenamed("Group.Solo", "Group_Solo")

### BillboardHot100

In [0]:
#bbHot100.display()

In [0]:
#bbHot100.printSchema()

In [0]:
##Eliminando las columnas "_c0" y "Unnamed: 0"
bbHot100 = bbHot100.drop("_c0")
bbHot100 = bbHot100.drop("Unnamed: 0")

In [0]:
# Cambiando de nombre a la columna Weekly.rank por Weeky_rank
bbHot100 = bbHot100.withColumnRenamed("Weekly.rank", "Weeky_rank")
# Cambiando de nombre a la columna Peak.position por Peek_position
bbHot100 = bbHot100.withColumnRenamed("Peak.position", "Peak_position")
# Cambiando de nombre a la columna Weeks.on.chart por Weeks_on_chart
bbHot100 = bbHot100.withColumnRenamed("Weeks.on.chart", "Weeks_on_chart")
bbHot100 = bbHot100.withColumnRenamed("Date", "Release_date")


In [0]:
#Casteando columna Week de str a date
bbHot100 = bbHot100.withColumn("Week", col("Week").cast("date")) 

#### Transformación columna Release_date de string a date

In [0]:
# Separando el año con ","
split_test = F.split(bbHot100['Release_date'], ',')
bbHot100 = bbHot100.withColumn('Rel_year', split_test.getItem(1))
bbHot100 = bbHot100.withColumn('Rel_mon_day', split_test.getItem(0))

In [0]:
# Separando el mes y dia con " "
split_test2 = F.split(bbHot100['Rel_mon_day'], ' ')
bbHot100 = bbHot100.withColumn('Rel_month', split_test2.getItem(0))
bbHot100 = bbHot100.withColumn('Rel_day', split_test2.getItem(1))

In [0]:
# función que recibe un dataframe, una columna a modificar y un diccionario con los nuevos valores los cuales va reemplazando acorde el mismo
def reemplazar_valores(df, nombre_columna, nuevos_valores):
    reemplazar_val = udf(lambda x: nuevos_valores.get(x, x))
    df = df.withColumn(nombre_columna, reemplazar_val(nombre_columna))
    return df

months_dict = {'January': '1', 'February': '2', 'March': '3', 'April': '4', 'May': '5', 'June': '6',
               'July': '7', 'August': '8', 'September': '9', 'October': '10', 'November': '11', 'December': '12'}

In [0]:
bbHot100 = reemplazar_valores(bbHot100, 'Rel_month', months_dict)

In [0]:
bbHot100 = bbHot100.withColumn("Release_date" , to_date(F.concat(col("Rel_year"), F.lit("-"),col("Rel_month"), F.lit("-"), col("Rel_day"))))

In [0]:
#bbHot100.display()

In [0]:
bbHot100 = bbHot100.drop('Rel_year', 'Rel_mon_day', 'Rel_month', 'Rel_day')

In [0]:
# Reemplazando todos los nulos por "0" o "" respectivamente
bbHot100 = bbHot100.fillna(value = 0).fillna("")

In [0]:
#Casteando columna Release_date de str a date
bbHot100 = bbHot100.withColumn("Release_date", col("Release_Date").cast("date")) 

In [0]:
# Borrando registros duplicados si los hay
bbHot100 = bbHot100.dropDuplicates()

In [0]:
#bbHot100.printSchema()

### GrammyAlbums

In [0]:
#grammyAlbums.display()

In [0]:
#grammyAlbums.printSchema()

In [0]:
#Eliminando la columna "_c0"
grammyAlbums = grammyAlbums.drop("_c0") 

In [0]:
# Reemplazando todos los nulos por "0" o "" respectivamente
grammyAlbums = grammyAlbums.fillna(value = 0).fillna("")

In [0]:
# Borrando registros duplicados si los hay
grammyAlbums = grammyAlbums.dropDuplicates()

### GrammySongs

In [0]:
#grammySongs.display()

In [0]:
#grammySongs.printSchema()

In [0]:
#Eliminando las columnas "_c0" y "X"
grammySongs = grammySongs.drop("_c0")
grammySongs = grammySongs.drop("X")

In [0]:
# Reemplazando todos los nulos por "0" o "" respectivamente
grammySongs = grammySongs.fillna(value = 0).fillna("")

In [0]:
# Borrando registros duplicados si los hay
grammySongs = grammySongs.dropDuplicates()

### riaaAlbumCerts

In [0]:
#riaaAlbumCerts.display()

In [0]:
riaaAlbumCerts.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Album: string (nullable = true)
 |-- Artist: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Label: string (nullable = true)



In [0]:
#Eliminando la columna "_c0"
riaaAlbumCerts = riaaAlbumCerts.drop("_c0")

In [0]:
# Reemplazando todos los nulos por "0" o "" respectivamente
riaaAlbumCerts = riaaAlbumCerts.fillna(value = 0).fillna("")

In [0]:
# Borrando registros duplicados si los hay
riaaAlbumCerts = riaaAlbumCerts.dropDuplicates()

### riaaSingleCerts:

In [0]:
#riaaSingleCerts.display()

In [0]:
#riaaSingleCerts.printSchema()

In [0]:
#Eliminando la columna "X"
riaaSingleCerts = riaaSingleCerts.drop("X")

In [0]:
# Reemplazando todos los nulos por "0" o "" respectivamente
riaaSingleCerts = riaaSingleCerts.fillna(value = 0).fillna("")

In [0]:
# Borrando registros duplicados si los hay
riaaSingleCerts = riaaSingleCerts.dropDuplicates()

### songAttributes:

In [0]:
#songAttributes.display()

In [0]:
#songAttributes.printSchema()

In [0]:
#Eliminando la columna "_c0" y la columna "Mode" que no aporta al análisis
songAttributes = songAttributes.drop("_c0")
songAttributes = songAttributes.drop("Mode")

In [0]:
# Casteando columnas que pueden resultar de interes para futuros análisis
songAttributes = songAttributes.withColumn("Acousticness", col("Acousticness").cast("float"))
songAttributes = songAttributes.withColumn("Danceability", col("Danceability").cast("float"))
songAttributes = songAttributes.withColumn("Duration", col("Duration").cast("int"))
songAttributes = songAttributes.withColumn("Energy", col("Energy").cast("float"))
songAttributes = songAttributes.withColumn("Instrumentalness", col("Instrumentalness").cast("float"))
songAttributes = songAttributes.withColumn("Liveness", col("Liveness").cast("float"))
songAttributes = songAttributes.withColumn("Loudness", col("Loudness").cast("float"))
songAttributes = songAttributes.withColumn("Popularity", col("Popularity").cast("int"))
songAttributes = songAttributes.withColumn("Tempo", col("Tempo").cast("float"))
songAttributes = songAttributes.withColumn("TimeSignature", col("TimeSignature").cast("int"))
songAttributes = songAttributes.withColumn("Valence", col("Valence").cast("float"))


In [0]:
# Reemplazando todos los nulos por "0" o "" respectivamente
songAttributes = songAttributes.fillna(value = 0).fillna("")

In [0]:
# Borrando registros duplicados si los hay
songAttributes = songAttributes.dropDuplicates()

### spotifyWeeklyTop200:

In [0]:
#spotifyWeeklyTop200.display()

In [0]:
#Eliminando la columna "_c0"
spotifyWeeklyTop200 = spotifyWeeklyTop200.drop("_c0")

In [0]:
# Casteando columnas de interes
spotifyWeeklyTop200 = spotifyWeeklyTop200.withColumn("Streams", col("Streams").cast("int"))
spotifyWeeklyTop200 = spotifyWeeklyTop200.withColumn("Week", col("Week").cast("date"))

In [0]:
# Reemplazando todos los nulos por "0" o "" respectivamente
spotifyWeeklyTop200 = spotifyWeeklyTop200.fillna(value = 0).fillna("")

In [0]:
# Borrando registros duplicados si los hay
spotifyWeeklyTop200 = spotifyWeeklyTop200.dropDuplicates()

In [0]:
#spotifyWeeklyTop200.printSchema()

root
 |-- Name: string (nullable = false)
 |-- Artist: string (nullable = false)
 |-- Features: string (nullable = false)
 |-- Streams: integer (nullable = true)
 |-- Week: date (nullable = true)



## Guardado de los DF en Trusted Data

In [0]:
trusted_data = "abfss://integrador-longas-elias@dataintegrador.dfs.core.windows.net/trusted-data"

In [0]:
artist.write.save(f"{trusted_data}/azuresqldb/dbo.artistDf", format="parquet", mode="overwrite")
bbHot100.write.save(f"{trusted_data}/azuresqldb/dbo.billboardHot100_1999_2019", format="parquet", mode="overwrite") 
grammyAlbums.write.save(f"{trusted_data}/azuresqldb/dbo.grammyAlbums_1999_2019", format="parquet", mode="overwrite") 
grammySongs.write.save(f"{trusted_data}/azuresqldb/dbo.grammySongs_1999_2019", format="parquet", mode="overwrite") 
riaaAlbumCerts.write.save(f"{trusted_data}/azuresqldb/dbo.riaaAlbumCerts_1999_2019", format="parquet", mode="overwrite") 
riaaSingleCerts.write.save(f"{trusted_data}/azuresqldb/dbo.riaaSingleCerts_1999_2019", format="parquet", mode="overwrite") 
songAttributes.write.save(f"{trusted_data}/azuresqldb/dbo.songAttributes_1999_2019", format="parquet", mode="overwrite") 
spotifyWeeklyTop200.write.save(f"{trusted_data}/azuresqldb/dbo.spotifyWeeklyTop200Streams", format="parquet", mode="overwrite")